# Tensorflow模型训练

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


print('TensorFlow Version:', tf.__version__)
for device in tf.config.list_physical_devices('GPU'):
    print(device)

TensorFlow Version: 2.2.0
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')


## 模型自动求导

In [9]:
class MyModel(tf.keras.Model):
    
    def __init__(self, num_classes=3):
        super(MyModel, self).__init__(name='MyModel')
        self.dense1 = tf.keras.layers.Dense(32, activation='relu')
        self.dense2 = tf.keras.layers.Dense(num_classes)
        
    def call(self, inputs):
        outputs = self.dense1(inputs)
        return self.dense2(outputs)
    
iris = load_iris()
x, y = iris.data.astype('float32'), iris.target
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

model = MyModel(num_classes=3)
criterion = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

with tf.GradientTape() as tape:
    y_pred = model(x_train)
    loss = criterion(y_train, y_pred)
grads = tape.gradient(loss, model.trainable_variables)
optimizer.apply_gradients(zip(grads, model.trainable_variables))

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>

## 自定义训练模型

In [20]:
class MyModel(tf.keras.Model):
    
    def __init__(self, num_classes=3):
        super(MyModel, self).__init__(name='MyModel')
        self.dense1 = tf.keras.layers.Dense(32, activation='relu')
        self.dense2 = tf.keras.layers.Dense(num_classes)
        
    def call(self, inputs):
        outputs = self.dense1(inputs)
        return self.dense2(outputs)
    
iris = load_iris()
x, y = iris.data.astype('float32'), iris.target

batch_size = 64
train_set = tf.data.Dataset.from_tensor_slices((x, y))
train_set = train_set.shuffle(buffer_size=1024).batch(batch_size)
train_set

<BatchDataset shapes: ((None, 4), (None,)), types: (tf.float32, tf.int64)>

In [21]:
model = MyModel(num_classes=3)
criterion = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()
epochs = 10

for epoch in range(epochs):
    print(f'Start of Epoch {epoch}')
    
    for step, (inputs, targets) in enumerate(train_set):
        with tf.GradientTape() as tape:
            outputs = model(inputs)
            loss = criterion(targets, outputs)
        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        print(f'Step {step}, Loss: {float(loss)}')
        

Start of Epoch 0
Step 0, Loss: 2.2453317642211914
Step 1, Loss: 1.9430198669433594
Step 2, Loss: 2.247148036956787
Start of Epoch 1
Step 0, Loss: 2.0725715160369873
Step 1, Loss: 1.8647223711013794
Step 2, Loss: 1.7192860841751099
Start of Epoch 2
Step 0, Loss: 1.8021800518035889
Step 1, Loss: 1.779268741607666
Step 2, Loss: 1.5998104810714722
Start of Epoch 3
Step 0, Loss: 1.5538560152053833
Step 1, Loss: 1.7382197380065918
Step 2, Loss: 1.4419684410095215
Start of Epoch 4
Step 0, Loss: 1.4901008605957031
Step 1, Loss: 1.5317816734313965
Step 2, Loss: 1.4127180576324463
Start of Epoch 5
Step 0, Loss: 1.4345848560333252
Step 1, Loss: 1.398844838142395
Step 2, Loss: 1.323042631149292
Start of Epoch 6
Step 0, Loss: 1.2679696083068848
Step 1, Loss: 1.343008279800415
Step 2, Loss: 1.4194293022155762
Start of Epoch 7
Step 0, Loss: 1.3013818264007568
Step 1, Loss: 1.2548255920410156
Step 2, Loss: 1.2498396635055542
Start of Epoch 8
Step 0, Loss: 1.2479866743087769
Step 1, Loss: 1.21407938003

## 自定义训练模型进阶

In [41]:
class MyModel(tf.keras.Model):
    
    def __init__(self, num_classes=3):
        super(MyModel, self).__init__(name='MyModel')
        self.dense1 = tf.keras.layers.Dense(32, activation='relu')
        self.dense2 = tf.keras.layers.Dense(num_classes)
        
    def call(self, inputs):
        outputs = self.dense1(inputs)
        return self.dense2(outputs)
    
iris = load_iris()
x, y = iris.data.astype('float32'), iris.target
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.15)

batch_size = 16
train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_set = train_set.shuffle(buffer_size=1024).batch(batch_size)
valid_set = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
valid_set = valid_set.batch(batch_size)

In [42]:
criterion = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()
train_metric = tf.keras.metrics.SparseCategoricalAccuracy()
valid_metric = tf.keras.metrics.SparseCategoricalAccuracy()
model = MyModel(num_classes=3)
epochs = 20

for epoch in range(epochs):
    
    for step, (inputs, targets) in enumerate(train_set):
        with tf.GradientTape() as tape:
            outputs = model(inputs)
            loss = criterion(targets, outputs)
        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        train_metric(targets, outputs)
    print(f'Train Accuracy: {float(train_metric.result())}')    
    train_metric.reset_states()
    
    for step, (inputs, targets) in enumerate(valid_set):
        outputs = model(inputs)
        valid_metric(targets, outputs)
    print(f'Valid Accuracy: {float(valid_metric.result())}')
    valid_metric.reset_states()


Train Accuracy: 0.3700787425041199
Valid Accuracy: 0.1304347813129425
Train Accuracy: 0.3700787425041199
Valid Accuracy: 0.1304347813129425
Train Accuracy: 0.3700787425041199
Valid Accuracy: 0.17391304671764374
Train Accuracy: 0.4251968562602997
Valid Accuracy: 0.5652173757553101
Train Accuracy: 0.5590550899505615
Valid Accuracy: 0.5652173757553101
Train Accuracy: 0.4960629940032959
Valid Accuracy: 0.43478259444236755
Train Accuracy: 0.4251968562602997
Valid Accuracy: 0.260869562625885
Train Accuracy: 0.3858267664909363
Valid Accuracy: 0.21739129722118378
Train Accuracy: 0.3779527544975281
Valid Accuracy: 0.17391304671764374
Train Accuracy: 0.3858267664909363
Valid Accuracy: 0.260869562625885
Train Accuracy: 0.3937007784843445
Valid Accuracy: 0.260869562625885
Train Accuracy: 0.4409448802471161
Valid Accuracy: 0.3913043439388275
Train Accuracy: 0.6062992215156555
Valid Accuracy: 0.6086956262588501
Train Accuracy: 0.6929134130477905
Valid Accuracy: 0.6086956262588501
Train Accuracy: 0.7